## Download and install dependencies

In [ ]:
import shutil
import os

# Create the directory, including any necessary parent directories
os.makedirs('/kaggle/working/lmms-finetune', exist_ok=True)

%cd /kaggle/working
!git clone https://github.com/thisisAranya/lmms-finetune.git

In [ ]:
%cd /kaggle/working/lmms-finetune

In [ ]:
!python -m pip install -r requirements.txt

In [ ]:
!wandb login 'paste your wandb code here'

## Finetune script

In [ ]:
!torchrun $DISTRIBUTED_ARGS train.py \
    --model_id 'original model id' \
    --data_path 'your json datapath' \
    --eval_data_path '' \
    --image_folder 'image folder datapath' \
    --video_folder '' \
    --num_frames 8 \
    --output_dir ./checkpoints/llava_1.5_7blora16 \
    --report_to wandb \
    --run_name llava7blora32 \
    --deepspeed ./ds_configs/zero2.json \
    --bf16 False \
    --num_train_epochs 1 \
    --per_device_train_batch_size 2 \
    --per_device_eval_batch_size 1 \
    --gradient_accumulation_steps 1 \
    --eval_strategy "epoch" \
    --save_strategy "epoch" \
    --save_total_limit 1 \
    --learning_rate 5e-6 \
    --weight_decay 0. \
    --warmup_ratio 0.03 \
    --lr_scheduler_type "cosine" \
    --logging_steps 1 \
    --tf32 False \
    --model_max_length 512 \
    --gradient_checkpointing True \
    --dataloader_num_workers 2 \
    --train_vision_encoder False \
    --use_vision_lora False \
    --train_vision_projector False \
    --use_lora True \
    --q_lora True \
    --lora_r 32 \
    --lora_alpha 16

## Run adapter with base model

In [8]:
import torch
from transformers import AutoProcessor, LlavaForConditionalGeneration, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
)

original_model_id = "original model path"
model_id = "./checkpoints/llava_1.5_7blora16"

model = LlavaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    quantization_config=bnb_config,
    low_cpu_mem_usage=True,
)

processor = AutoProcessor.from_pretrained(original_model_id)

## Push the adapter to huggingface

In [9]:
!huggingface-cli login --token 'paste your huggingface token here'

In [10]:
model.push_to_hub("huggingface repo for your model", use_auth_token=True)

## Merge adapter with base model

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer,AutoProcessor, AutoModelForPreTraining, LlavaForConditionalGeneration
from transformers import pipeline
from peft import PeftModel, PeftConfig

org_model_id = "original model id path"

model = LlavaForConditionalGeneration.from_pretrained(
    org_model_id, 
    torch_dtype=torch.float16,
)

processor = AutoProcessor.from_pretrained(org_model_id)

model = PeftModel.from_pretrained(model,"adapter's path")

In [13]:
model = model.merge_and_unload()

## Push the merged model to huggingface

In [ ]:
!huggingface-cli login --token 'paste your huggingface token here'

In [ ]:
model.push_to_hub("huggingface repo for your merged model", use_auth_token=True)
processor.push_to_hub("huggingface repo for your merged model", use_auth_token=True)